<div style="width:100%; background-color: #121017;"><a target="_blank" href="http://university.yugabyte.com?utm_source=gitpod&utm_medium=notebook"><img src="assets/YBU_Logo.png" /></a></div><br>

> **Migrate and Modernize with YugabyteDB Voyager**
>
> Enroll for free at  [Yugabyte University](https://university.yugabyte.com/courses/YugabyteDB-Voyager?utm_source=gitpod&utm_medium=notebook).
>

<br>
This notebook file is:

`07_Migrate_to_YugabyteDB_Managed.ipynb`

## 🛠️ Requirements
Here are the requirements for this notebook:
- ✅ Create the notebook variables in `01_Lab_Setup.ipynb`, which you previously did
- ✅ Create the `db_ybu` database, which you previously did
- ☑️ A Python 3.12 or higher kernel for the notebook, *which you need to select right now!!!*
- ☑️ Import the notebook variables, *which you must do next*
- ☑️ Create a YugabyteDB Managed free, sandbox cluster at [https://cloud.yugabyte.com](https://cloud.yugabyte.com), *which you must do next*
- ☑️ Prepare, Migrate schema, and Migrate data, *which you must do next*


### Select your notebook kernel
- In the Notebook toolbar, click **Select Kernel**.
<br>
<img width=50% src="assets/01_01_Select_Kernel_Toolbar.png" />

- Next, in the dropdown, select **Python 3.12** or higher.
<br>
<img width=50% src="assets/01_02_Select_Kernel_Dropdown.png" />

That's it!

## ⛑️ Getting help
The best way to get help us from the Yugabyte University team. You can post your question in YugabyteDB Community Slack in the #training or #yb-university channels. Join our [community](https://communityinviter.com/apps/yugabyte-db/register).

## 👣 Set up steps for this notebook
Here are the steps to setup this lab:

- Import the notebook variables

### 👇 Import the notebook variables

> 👉 IMPORTANT! 👈
> 
> Do **NOT** skip running the following cell. 
> 

The following Python cell reads the stored variables created in the `01_Lab_Setup.ipynb` notebook. To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell. 

👇 👇 👇 

In [ ]:
# Use %store -r to read 01_Lab_Setup variables
%store -r 
%config SqlMagic.autopandas=False
# %config SqlMagic.named_parameters=True
%config SqlMagic.displaylimit=30
%config SqlMagic.displaycon=False

**Update the notebook styling**

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

### Get Your Gitpod to the IP address
Run the following to determine the public ip address of your Gitpod host.

In [ ]:
%%bash --out MY_GITPOD_IP_ADDRESS
MY_GITPOD_IP_ADDRESS=`curl -s https://ipinfo.io/ip`
echo $MY_GITPOD_IP_ADDRESS

> No IP Address ???
>
> If you don't get an IP address, rerun the above again! It may make a few tries!

Store your address as a notebook variable.

In [ ]:
%store MY_GITPOD_IP_ADDRESS
print(MY_GITPOD_IP_ADDRESS)

> No output ???
>
> Rerun the above two cells and confirm that the output of the IP address.

---
## Prepare YugabyteDB Managed
- Create a sandbox cluster
- Create a New IP Allow List and add your Gitpod IP to the IP Allow List
- Download the `root.crt` file
- Copy the DNS Host name
- Create a target database and Voyager user

### Create a sandbox cluster 

Create a YugabyteDB Managed free, sandbox cluster at [https://cloud.yugabyte.com](https://cloud.yugabyte.com/utm_source=gitpod&utm_medium=notebook). 
- Sign up or Sign in
- Select **Create a Free** cluster.
- In **Choose a cluster type**, select the **Sandbox** card, and select **Choose**.
- In **(1) Cluster Settings**...
  - In **Cluster Name**, enter `migrate-mysql`.
  - Select a provide such as AWS or GCP.
  - Select a region.
  - Select **Next**.
- In **(2) Network Access**...
  - Select **Create New IP Allow List**.
  - In the **Create New IP Allow List** dialog...
    - For **IP ALLOW LIST NAME**, enter `ybu-yugabytedb-voyager-gitpod`
    - In **IP ADDRESS(ES) OR CIDR RANGE**, copy the output value of the bash cell above this one, e.g, `$MY_GITPOD_IP_ADDRESS`.
    - Select **Create**.
  - Select **Next**.
- In **(3) DB Credentials**...
  - Select **Add your own credentials**
  - For **User**, enter `admin`.
  - For **Password** and **Confirm Password**, enter `Yugabyte#1`.
  - Select **Download Credentials**.
  - Select **Create Cluster**.

While the cluster is spinning up, which may take about 5 minutes, store your User and Password as notebook variables. The cell below has the default values in the instructions above.

In [ ]:
MY_YUGABYTEDB_MANAGED_YSQL_USER = 'admin'
MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD = 'Yugabyte#1'

%store MY_YUGABYTEDB_MANAGED_YSQL_USER
%store MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD

print(MY_YUGABYTEDB_MANAGED_YSQL_USER)
print(MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD)

Wait until your YugabyteDB Managed cluster is running.

Download the `root.crt` file.
- In the Cluster dashboard, select **Connect**.
- In the **Connect to Cluster** dialog, in **Connection Methods**, in **Connect to your Application**, select **View Guide**.
- In the **Connect to Cluster** dialog, in **1) Download the CA certificate...**, select **Download CA Cert**.
- Drag and drop the `root.crt` file into the `\tmp` directory in the Explorer panel, here in this Gitpod VS Code environment. 

> 👉 IMPORTANT! 👈
> Copy your `root.crt` to the `\tmp` directory in the file explorer of the Gitpod environment. You can literally drag and drop the file to the `\tmp` in the Explorer panel. 
> 
> There is file already there that you can ignore, `nada.txt`.
>

- Keep the dialog open as you will need to copy parameter values.

Verify that you have the `root.crt` file and that this notebook can read it.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}
echo $NOTEBOOK_VOYAGER_TMP
cat  $NOTEBOOK_VOYAGER_TMP/root.crt

Next, copy the Host parameter  and assign the values to a notebook variable.
-  In the **Connect to Cluster** dialog, in **2) Use the following parameters...**, select **YSQL**.
-  Select the **Parameters** tab.
-  For Host, select the Copy icon.
-  Store the Host value in the following notebook variable.

In [ ]:
MY_YUGABYTEDB_MANAGED_HOST = 'ADD_HOST_HERE';
%store MY_YUGABYTEDB_MANAGED_HOST
print(MY_YUGABYTEDB_MANAGED_HOST)

> 👉 IMPORTANT! 👈
>
> Make sure that you copy the Host for your YugabyteDB Managed sandbox cluster. 
> 
> It should be something like: <br>
> `us-east-1.fb67380f-e7d8-4709-8fe0-5a7182146558.aws.ybdb.io`

- Close the **Connect to Cluster** dialog.

### Create a target database and Voyager user

Create the target database in your YugabyteDB cluster. The database name can be the same or different from the source database name. If the target database name is not provided, yb-voyager assumes the target database name to be yugabyte. If you do not want to import in the default yugabyte database, specify the name of the target database name using the --target-db-name argument to the yb-voyager import commands.

```sql
CREATE DATABASE target_db_name;
```

```sql
CREATE USER ybvoyager PASSWORD 'password';
GRANT yb_superuser TO ybvoyager;
```


> 👉 IMPORTANT! 👈
> 
> Copy your `root.crt` to the `\tmp` directory in the file explorer of the Gitpod environment. You can literally drag and drop the file to the `\tmp` in the Explorer panel. 

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME" "$NB_NOTEBOOK_VOYAGER_TMP" "$MY_YUGABYTEDB_MANAGED_HOST" "$MY_YUGABYTEDB_MANAGED_YSQL_USER" "$MY_YUGABYTEDB_MANAGED_YSQL_PASSWORD"
YB_PATH_BIN=${1}
DB_NAME=${2}
NOTEBOOK_VOYAGER_TMP=${3}
YUGABYTEDB_MANAGED_HOST=${4}
YUGABYTEDB_MANAGED_YSQL_USER=${5}
YUGABYTEDB_MANAGED_YSQL_PASSWORD=${6}

cd $YB_PATH_BIN

MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_HOST_PORT=${YUGABYTEDB_MANAGED_HOST}:${MY_YUGABYTEDB_MANAGED_PORT}
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt
MY_YUGABYTEDB_MANAGED_CONN="postgresql://${YUGABYTEDB_MANAGED_YSQL_USER}:${YUGABYTEDB_MANAGED_YSQL_PASSWORD}@${MY_YUGABYTEDB_MANAGED_HOST_PORT}/yugabyte?ssl=true&sslmode=verify-full&sslrootcert=${MY_YUGABYTEDB_MANAGED_CRT}"
echo $MY_YUGABYTEDB_MANAGED_CONN

./ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "drop database if exists "${DB_NAME}";"
./ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "drop database if exists ybvoyager_metadata;" 
./ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "create database "${DB_NAME}";"
./ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "drop user if exists ybvoyager; create user ybvoyager PASSWORD 'Yugabyte#1'; GRANT yb_superuser TO ybvoyager;" 


Verify that the `ybvoyager` user can connect.


In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME" "$NB_NOTEBOOK_VOYAGER_TMP" "$MY_YUGABYTEDB_MANAGED_HOST" 
YB_PATH_BIN=${1}
DB_NAME=${2}
NOTEBOOK_VOYAGER_TMP=${3}
YUGABYTEDB_MANAGED_HOST=${4}

cd $YB_PATH_BIN

MY_YUGABYTEDB_MANAGED_VOYAGER_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD=Yugabyte#1

MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_HOST_PORT=${YUGABYTEDB_MANAGED_HOST}:${MY_YUGABYTEDB_MANAGED_PORT}
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt
MY_YUGABYTEDB_MANAGED_CONN="postgresql://${MY_YUGABYTEDB_MANAGED_VOYAGER_USER}:${MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD}@${MY_YUGABYTEDB_MANAGED_HOST_PORT}/${DB_NAME}?ssl=true&sslmode=verify-full&sslrootcert=${MY_YUGABYTEDB_MANAGED_CRT}"
echo $MY_YUGABYTEDB_MANAGED_CONN

./ysqlsh $MY_YUGABYTEDB_MANAGED_CONN  -c "select 1 as yes_connected;" 
./ysqlsh $MY_YUGABYTEDB_MANAGED_CONN  -c "\l" 

---
## Import modified schema

In the previous notebook, you ran a cell that ended the migration. In doing so, you also copied the original export directory. You will use this copy of the export directory for the migration to your YugabyteDB Managed cluster.

In [ ]:
%%bash -s "$NB_DB_NAME" "$NB_NOTEBOOK_VOYAGER_TMP" "$MY_YUGABYTEDB_MANAGED_HOST" 
DB_NAME=${1}
NOTEBOOK_VOYAGER_TMP=${2}
YUGABYTEDB_MANAGED_HOST=${3}

MY_YUGABYTEDB_MANAGED_VOYAGER_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD=Yugabyte#1

MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt


yb-voyager import schema  -y  \
  --start-clean true \
  --export-dir $NOTEBOOK_VOYAGER_TMP/my-export-dir \
  --target-db-host ${YUGABYTEDB_MANAGED_HOST} \
  --target-db-port  ${MY_YUGABYTEDB_MANAGED_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user ${MY_YUGABYTEDB_MANAGED_VOYAGER_USER} \
  --target-db-password ${MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD} \
  --target-ssl-mode "verify-full" \
  --target-ssl-root-cert ${MY_YUGABYTEDB_MANAGED_CRT} \
  --target-db-schema "sakila" > /dev/null
  

Verify the schema creation in your YugabyteDB Managed cluster.

In [ ]:
%%bash -s "$NB_YB_PATH_BIN" "$NB_DB_NAME" "$NB_NOTEBOOK_VOYAGER_TMP" "$MY_YUGABYTEDB_MANAGED_HOST" 
YB_PATH_BIN=${1}
DB_NAME=${2}
NOTEBOOK_VOYAGER_TMP=${3}
YUGABYTEDB_MANAGED_HOST=${4}

cd $YB_PATH_BIN

MY_YUGABYTEDB_MANAGED_VOYAGER_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD=Yugabyte#1

MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_HOST_PORT=${YUGABYTEDB_MANAGED_HOST}:${MY_YUGABYTEDB_MANAGED_PORT}
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt
MY_YUGABYTEDB_MANAGED_CONN="postgresql://${MY_YUGABYTEDB_MANAGED_VOYAGER_USER}:${MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD}@${MY_YUGABYTEDB_MANAGED_HOST_PORT}/${DB_NAME}?ssl=true&sslmode=verify-full&sslrootcert=${MY_YUGABYTEDB_MANAGED_CRT}"
echo $MY_YUGABYTEDB_MANAGED_CONN

./ysqlsh $MY_YUGABYTEDB_MANAGED_CONN -c "\dn" 

---
## Import source data
Import the source data to the target YugabyteDB database using the `yb-voyager import data` command.


In [ ]:
%%bash -s "$NB_DB_NAME" "$NB_NOTEBOOK_VOYAGER_TMP" "$MY_YUGABYTEDB_MANAGED_HOST" 
DB_NAME=${1}
NOTEBOOK_VOYAGER_TMP=${2}
YUGABYTEDB_MANAGED_HOST=${3}


MY_YUGABYTEDB_MANAGED_VOYAGER_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD=Yugabyte#1
MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt


##update to use connection string, user, and SSL?
yb-voyager import data -y \
  --start-clean true \
  --export-dir $NOTEBOOK_VOYAGER_TMP/my-export-dir \
  --target-db-host ${YUGABYTEDB_MANAGED_HOST} \
  --target-db-port  ${MY_YUGABYTEDB_MANAGED_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user ${MY_YUGABYTEDB_MANAGED_VOYAGER_USER} \
  --target-db-password ${MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD} \
  --target-ssl-mode "verify-full" \
  --target-ssl-root-cert ${MY_YUGABYTEDB_MANAGED_CRT} \
  --target-db-schema "sakila" 

You can check the status of the import.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP"
NOTEBOOK_VOYAGER_TMP=${1}

yb-voyager import data status \
  --export-dir $NOTEBOOK_VOYAGER_TMP/my-export-dir


---
## Post Import Data
After importing the data, rerun the import schema command, but this time, using the arguments, `post-import-data` and `refresh-mviews`.


> 👉 IMPORTANT! 👈
> - Change the values in the variables for `ADD_HOST_HERE` 
> - Copy your `root.crt` to the `\tmp` directory in the file explorer


In [ ]:
%%bash -s "$NB_DB_NAME" "$NB_NOTEBOOK_VOYAGER_TMP" "$MY_YUGABYTEDB_MANAGED_HOST" 
DB_NAME=${1}
NOTEBOOK_VOYAGER_TMP=${2}
YUGABYTEDB_MANAGED_HOST=${3}


MY_YUGABYTEDB_MANAGED_VOYAGER_USER=ybvoyager
MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD=Yugabyte#1
MY_YUGABYTEDB_MANAGED_PORT=5433
MY_YUGABYTEDB_MANAGED_CRT=${NOTEBOOK_VOYAGER_TMP}/root.crt


yb-voyager import schema  -y \
  --post-import-data true \
  --refresh-mviews true \
  --export-dir $NOTEBOOK_VOYAGER_TMP/my-export-dir \
  --target-db-host ${YUGABYTEDB_MANAGED_HOST} \
  --target-db-port  ${MY_YUGABYTEDB_MANAGED_PORT} \
  --target-db-name  ${DB_NAME} \
  --target-db-user ${MY_YUGABYTEDB_MANAGED_VOYAGER_USER} \
  --target-db-password ${MY_YUGABYTEDB_MANAGED_VOYAGER_PASSWORD} \
  --target-ssl-mode "verify-full" \
  --target-ssl-root-cert ${MY_YUGABYTEDB_MANAGED_CRT} \
  --target-db-schema "sakila"

---
## Verify migration
After completing the migration of the schema and the data,the automated migration is finished. Next is the manual verification of the migration success. 

You need to manually validate the successful migration of database objects, including the post import data objects such as triggers and indexes.

You also need to manually run validation queries on both the source and target database to ensure that the data is correctly migrated. A sample query to validate the databases can include checking the row count of each table.


### Connect to your cluster in YugabyteDB Managed using the PostgreSQL Driver for Python

> 👉 IMPORTANT! 👈
> - Change the values in the variables for `ADD_HOST_HERE`.

Copy the connection string from the above cell output into `conn_str`

In [ ]:
# Connect to db_ybu in YugabyteDB Managed, requires you to update `ADD_HOST_HERE`

import psycopg2
import sqlalchemy as alc
from sqlalchemy import create_engine


db_name=NB_DB_NAME

managed_host=MY_YUGABYTEDB_MANAGED_HOST

connection_managed_str='postgresql://ybvoyager:Yugabyte#1@'+managed_host+':5433/'+db_name
print(connection_managed_str)
engine = create_engine(connection_managed_str)

#### Load SQL magic extension
>IMPORTANT!
>
> To use SQL magic, you must run the following cell that loads the notebook extension.

In [ ]:
%reload_ext sql
# creates connection for sql magic
%sql {connection_managed_str}

### Run queries to validate objects

Verify the creation of indexes.

In [ ]:
%%sql
select tablename, indexname, indexdef from pg_indexes where schemaname = 'sakila' order by tablename, indexname;

Verify the creation of triggers.

In [ ]:
%%sql
select  event_object_table AS table_name, trigger_name from information_schema.triggers where trigger_schema = 'sakila' group by table_name, trigger_name order by table_name, trigger_name;

### Run queries

In [ ]:
%%sql
select * from sakila.actor where UPPER(first_name) = 'SCARLETT';

In [ ]:
%%sql
select count(distinct last_name) from sakila.actor;

In [ ]:
%%sql
select actor.actor_id, actor.first_name, actor.last_name,
       count(actor_id) as film_count
from sakila.actor join sakila.film_actor using (actor_id)
group by actor_id
order by film_count desc
limit 1;

---
## End the migration

Run the `end migration` command.

In [ ]:
%%bash -s "$NB_NOTEBOOK_VOYAGER_TMP" "$TARGET_DB_PASSWORD"
NOTEBOOK_VOYAGER_TMP=${1}
export TARGET_DB_PASSWORD=${2}
#echo $NOTEBOOK_VOYAGER_TMP
#echo $TARGET_DB_PASSWORD
printenv TARGET_DB_PASSWORD

yb-voyager end migration -y \
  --export-dir $NOTEBOOK_VOYAGER_TMP/my-export-dir \
  --backup-dir $NOTEBOOK_VOYAGER_TMP/backup-dir \
  --backup-data-files false \
  --backup-schema-files false \
  --backup-log-files false \
  --save-migration-reports false 


---

## 😊 That's it!
Nicely done!!! 

You migrated again, this time from this Gitpod environment to your cluster in YugabyteDB Managed! 

Awesome !!!

You can now close this Gitpod browser and continue with the course at Yugabyte University.

> **Migrate and Modernize with YugabyteDB Voyager**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/courses/YugabyteDB-Voyager?utm_source=gitpod&utm_medium=notebook).
>
